In [284]:
from scipy.sparse import save_npz, load_npz, csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
import sklearn
import numpy as np
import pandas as pd
import pickle

### Загружаем подготовленную ранее разреженную матрицу (users, items)

In [4]:
train_set = load_npz('model_req/train_set_2023-05-27.npz')
train_set

<50785x23444 sparse matrix of type '<class 'numpy.float32'>'
	with 401071 stored elements in Compressed Sparse Row format>

### Сжимаем векторное пространство до 256 компонентов

In [306]:
svd = TruncatedSVD(n_components=256)
X_dense = svd.fit_transform(train_set)

### KNN на 128 ближайших соседей

In [307]:
num_neighbors = 128
knn = NearestNeighbors(n_neighbors=num_neighbors, metric='cosine')
knn.fit(X_dense)

NearestNeighbors(metric='cosine', n_neighbors=128)

### Подгружаем ранее заготовленные словари, для "расшифровки" работы модели 

In [329]:
with open('model_req/id_to_user.pickle', 'rb') as fp:
    id_to_user = pickle.load(fp)

with open('model_req/user_to_id.pickle', 'rb') as fp:
    user_to_id = pickle.load(fp)    

In [43]:
attend_df = pd.read_csv('datasets/attend.csv')
attend_df.head()

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00


### Пример работы модели

In [330]:
row_dense = svd.transform(train_set[user_to_id[101346552]])
results = knn.kneighbors(row_dense, num_neighbors)

results

(array([[0.        , 0.3281362 , 0.32823992, 0.32938755, 0.33296984,
         0.33374304, 0.3359201 , 0.3505534 , 0.35254216, 0.356645  ,
         0.35775393, 0.36317456, 0.36633158, 0.36840177, 0.36872262,
         0.37098092, 0.37253886, 0.37280202, 0.37323058, 0.3749122 ,
         0.3760413 , 0.37620807, 0.37620807, 0.37620807, 0.3769377 ,
         0.37753332, 0.37763226, 0.37784624, 0.37793565, 0.37796414,
         0.37801814, 0.37839448, 0.37895715, 0.3791883 , 0.3810054 ,
         0.3828938 , 0.3856423 , 0.3865556 , 0.3871777 , 0.38747847,
         0.3956871 , 0.3966517 , 0.39762795, 0.39767194, 0.3982473 ,
         0.40167135, 0.40271038, 0.40361023, 0.40478277, 0.405146  ,
         0.40842658, 0.41098338, 0.41438097, 0.4178416 , 0.41794372,
         0.41963655, 0.42606091, 0.42834628, 0.42834628, 0.42834628,
         0.42834628, 0.43151426, 0.43169928, 0.43247193, 0.43318307,
         0.43330884, 0.43586326, 0.43686724, 0.43711627, 0.43875253,
         0.44032407, 0.44548059, 0

In [301]:
attend_df[attend_df['уникальный номер участника'] == id_to_user[1701]]['направление 3'].unique()

array(['Зумба', 'ОНЛАЙН Восточные танцы', 'Калланетика',
       'ОНЛАЙН Танцы для всех',
       'ОНЛАЙН География. Путешествия вокруг света', 'Танцы для всех',
       'ОНЛАЙН Здорово жить', 'ОНЛАЙН История искусства', 'Хоровое пение',
       'ОНЛАЙН Литература', 'ИЗО', 'Различные техники рисования',
       'Краеведение и пешие прогулки',
       'ОНЛАЙН Краеведение и онлайн-экскурсии'], dtype=object)

In [302]:
attend_df[attend_df['уникальный номер участника'] == 101346998]['направление 3'].unique()

array(['ОНЛАЙН Суставная гимнастика', 'ОНЛАЙН История, культура Москвы',
       'ОНЛАЙН Краеведение и онлайн-экскурсии', 'Йога',
       'Роспись по дереву. Художественная обработка древесины',
       'Краеведение и пешие прогулки', 'История, культура Москвы',
       'ОНЛАЙН Здорово жить', 'ОНЛАЙН История, культура России',
       'ОНЛАЙН Танцы для всех',
       'ОНЛАЙН Краеведение и онлайн-экскурсии по Москве и России',
       'ОНЛАЙН История искусства', 'ОНЛАЙН Основы духовной культуры',
       'ОНЛАЙН Литература', 'ОНЛАЙН География. Путешествия вокруг света',
       'Английский язык',
       'ОНЛАЙН Мастер-класс по уходу за кожей в зрелом возрасте',
       'ОНЛАЙН Правильное питание', 'ОНЛАЙН Школа макияжа'], dtype=object)

### Сохраняем новый файлы

In [311]:
with open('model_req/knn.pickle', 'wb') as knnPickle:
    pickle.dump(knn, knnPickle)
    
with open('model_req/svd.pickle', 'wb') as svdPickle:
    pickle.dump(svd, svdPickle)